In [ ]:
!pip install psycopg2

In [ ]:
!pip install py7zr

In [50]:
import psycopg2
import sqlalchemy

import pandas as pd

from timeit import default_timer as timer
import time as tm
import sys

from ftplib import FTP
import os
import re
import py7zr

from io import StringIO

import glob

In [4]:
print(sys.version.split()[0])

3.10.9


In [27]:
# Verificando se o servidor FTP está ativo
server = "ftp.mtps.gov.br"

try:
    # Conecta ao servidor FTP
    ftp = FTP(server)
    # Faz login (opcional)
    # ftp.login("usuario", "senha")
    # Verifica se a conexão foi bem sucedida
    if ftp.getwelcome():
        print("Conexão bem sucedida ao servidor FTP:", server)
    # Fecha a conexão com o servidor FTP
    ftp.quit()

except Exception as e:
    print("Falha na conexão com o servidor FTP:", server)
    print(e)

Conexão bem sucedida ao servidor FTP: ftp.mtps.gov.br


In [4]:
path = 'C:/Users/thiago.silva/TCC/RAIS'
path_tmp = path+'/temp'

In [28]:
########################################################
# TRECHO DE CÓDIGO APENAS PARA O DOWNLOAD DOS ARQUIVOS #
########################################################

path = 'C:/Users/thiago.silva/TCC/RAIS'
path_tmp = path+'/temp'

ftp = FTP('ftp.mtps.gov.br')
ftp.login()
ftp.cwd('/pdet/microdados/RAIS')

anos = [ano for ano in range(2010,2022)]

# define o diretório local onde os arquivos serão salvos
local_dir = os.path.join(os.getcwd(), "dados_rais")

if not os.path.exists(local_dir):
    os.makedirs(local_dir)

for ano in anos:
    ftp.cwd(f'/pdet/microdados/RAIS/{ano}')
    
    print(f"Baixando arquivo de {ano}...")        
    for filename in ftp.nlst():
        if 'ESTAB' in filename or 'ESTB' in filename or 'Estb' in filename:
            year_match = re.search(r'\b(20\d{2})\b', filename)  # Encontra um padrão de quatro dígitos numéricos que representam o ano

            if year_match:
                year = year_match.group(1)  # Obtém o ano encontrado na string
            else:
                year = ano  # Substitua 'YEAR' pelo ano que você deseja adicionar caso não haja um ano no nome do arquivo

                # Adiciona o ano ao nome do arquivo
                new_filename = f"ESTB{year}.7z"

                local_path = os.path.join(path_tmp, new_filename)  # Substitua '/caminho/do/diretorio' pelo caminho desejado
                with open(local_path, 'wb') as file:
                    ftp.retrbinary('RETR ' + filename, file.write)

Baixando arquivo de 2010...
Baixando arquivo de 2011...
Baixando arquivo de 2012...
Baixando arquivo de 2013...
Baixando arquivo de 2014...
Baixando arquivo de 2015...
Baixando arquivo de 2016...
Baixando arquivo de 2017...
Baixando arquivo de 2018...
Baixando arquivo de 2019...
Baixando arquivo de 2020...
Baixando arquivo de 2021...


In [29]:
for filename in os.listdir(path_tmp):
    if filename.endswith('.7z'):
        arquivo_7z = os.path.join(path_tmp, filename)

        with py7zr.SevenZipFile(arquivo_7z, mode='r') as z:
            z.extractall(path=path)

        print(f"Arquivo {filename} descompactado para {path}")

        arquivos_txt = [f for f in os.listdir(path) if f.startswith('ESTB') or f.startswith('Estb') and f.endswith('.txt')]
        years = [int(arquivo_txt[4:8]) for arquivo_txt in arquivos_txt]
        last_year = max(years)
        
        arquivo = 'RAIS_ESTAB_PUB.txt'

        if os.path.exists(os.path.join(path, arquivo)):
            novo_nome_arquivo = f'ESTB{last_year + 1}.txt'
            caminho_novo_nome_arquivo = os.path.join(path, novo_nome_arquivo)
            os.rename(os.path.join(path, arquivo), caminho_novo_nome_arquivo)

        # Exclui o arquivo 7z após a descompactação
        os.remove(arquivo_7z)
        print(f"Arquivo {filename} excluído.")

Arquivo ESTB2010.7z descompactado para C:/Users/thiago.silva/TCC/RAIS
Arquivo ESTB2010.7z excluído.
Arquivo ESTB2011.7z descompactado para C:/Users/thiago.silva/TCC/RAIS
Arquivo ESTB2011.7z excluído.
Arquivo ESTB2012.7z descompactado para C:/Users/thiago.silva/TCC/RAIS
Arquivo ESTB2012.7z excluído.
Arquivo ESTB2013.7z descompactado para C:/Users/thiago.silva/TCC/RAIS
Arquivo ESTB2013.7z excluído.
Arquivo ESTB2014.7z descompactado para C:/Users/thiago.silva/TCC/RAIS
Arquivo ESTB2014.7z excluído.
Arquivo ESTB2015.7z descompactado para C:/Users/thiago.silva/TCC/RAIS
Arquivo ESTB2015.7z excluído.
Arquivo ESTB2016.7z descompactado para C:/Users/thiago.silva/TCC/RAIS
Arquivo ESTB2016.7z excluído.
Arquivo ESTB2017.7z descompactado para C:/Users/thiago.silva/TCC/RAIS
Arquivo ESTB2017.7z excluído.
Arquivo ESTB2018.7z descompactado para C:/Users/thiago.silva/TCC/RAIS
Arquivo ESTB2018.7z excluído.
Arquivo ESTB2019.7z descompactado para C:/Users/thiago.silva/TCC/RAIS
Arquivo ESTB2019.7z excluído.


In [30]:
# Ajustando nome do arquivo de 2012
antigo2012 = 'Estb2012.txt'
novo2012 = 'ESTB2012.txt'
os.rename(os.path.join(path, antigo2012), os.path.join(path, novo2012))

In [33]:
# Lista de arquivos extraídos
arquivos = [f for f in os.listdir(path) if f.startswith('ESTB') and f.endswith('.txt')]

for arquivo in arquivos:
    novo_nome_arquivo = arquivo[:-4] + ".csv"
    caminho_arquivo_original = os.path.join(path, arquivo)
    caminho_arquivo_renomeado = os.path.join(path, novo_nome_arquivo)
    os.rename(caminho_arquivo_original, caminho_arquivo_renomeado)
    print(f"Arquivo {arquivo} renomeado para {novo_nome_arquivo}.")

print("Todos os arquivos foram renomeados para a extensão .csv.")

Arquivo ESTB2010.txt renomeado para ESTB2010.csv.
Arquivo ESTB2011.txt renomeado para ESTB2011.csv.
Arquivo ESTB2012.txt renomeado para ESTB2012.csv.
Arquivo ESTB2013.txt renomeado para ESTB2013.csv.
Arquivo ESTB2014.txt renomeado para ESTB2014.csv.
Arquivo ESTB2015.txt renomeado para ESTB2015.csv.
Arquivo ESTB2016.txt renomeado para ESTB2016.csv.
Arquivo ESTB2017.txt renomeado para ESTB2017.csv.
Arquivo ESTB2018.txt renomeado para ESTB2018.csv.
Arquivo ESTB2019.txt renomeado para ESTB2019.csv.
Arquivo ESTB2020.txt renomeado para ESTB2020.csv.
Arquivo ESTB2021.txt renomeado para ESTB2021.csv.
Todos os arquivos foram renomeados para a extensão .csv.


In [39]:
arquivos = [f for f in os.listdir(path) if f.startswith('ESTB') and f.endswith('.csv')]

for arquivo in arquivos:
    ano = arquivo[4:8]
    caminho_arquivo = os.path.join(path, arquivo)
    df = pd.read_csv(caminho_arquivo, delimiter=';', encoding='cp1252')
    df['Ano'] = ano
    df.to_csv(caminho_arquivo, sep=';', index=False)

print("Coluna de ano adicionada em todos os arquivos CSV.")

C:\Users\thiago.silva\AppData\Local\Temp\ipykernel_23040\3418199052.py:6: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_arquivo, delimiter=';', encoding='cp1252')
C:\Users\thiago.silva\AppData\Local\Temp\ipykernel_23040\3418199052.py:6: DtypeWarning: Columns (3,4,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_arquivo, delimiter=';', encoding='cp1252')
C:\Users\thiago.silva\AppData\Local\Temp\ipykernel_23040\3418199052.py:6: DtypeWarning: Columns (3,4,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_arquivo, delimiter=';', encoding='cp1252')


Coluna de ano adicionada em todos os arquivos CSV.


In [42]:
todas_colunas = []
for arquivo in arquivos:
    caminho_arquivo = os.path.join(path, arquivo)
    df = pd.read_csv(caminho_arquivo, delimiter=';')
    colunas = df.columns.tolist()
    todas_colunas.extend(colunas)

# Remove as colunas repetidas
colunas_unicas = list(set(todas_colunas))

print("Colunas únicas de todos os arquivos CSV:")
print(colunas_unicas)

C:\Users\thiago.silva\AppData\Local\Temp\ipykernel_23040\142648608.py:4: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_arquivo, delimiter=';')
C:\Users\thiago.silva\AppData\Local\Temp\ipykernel_23040\142648608.py:4: DtypeWarning: Columns (3,4,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_arquivo, delimiter=';')
C:\Users\thiago.silva\AppData\Local\Temp\ipykernel_23040\142648608.py:4: DtypeWarning: Columns (3,4,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_arquivo, delimiter=';')


Colunas únicas de todos os arquivos CSV:
['CNAE 95 Classe', 'Ind CEI Vinculado', 'Ind Simples', 'IBGE Subsetor', 'Distritos SP', 'Bairros RJ', 'Regiões Adm DF', 'Qtd Vínculos Estatutários', 'Ind Estab Participa PAT', 'CEP Estab', 'Bairros SP', 'Qtd Vínculos Ativos', 'UF', 'Município', 'Bairros Fortaleza', 'CNAE 2.0 Classe', 'Natureza Jurídica', 'Ind Rais Negativa', 'Tipo Estab', 'Tipo Estab.1', 'Tamanho Estabelecimento', 'Ind Atividade Ano', 'CNAE 2.0 Subclasse', 'Ano', 'Qtd Vínculos CLT']


In [40]:
for arquivo in arquivos:
    caminho_arquivo = os.path.join(path, arquivo)
    df = pd.read_csv(caminho_arquivo, delimiter=';')
    colunas = df.columns.tolist()

    print(f"Colunas do arquivo {arquivo}:")
    print(colunas)

Colunas do arquivo ESTB2010.csv:
['Bairros SP', 'Bairros Fortaleza', 'Bairros RJ', 'CNAE 2.0 Classe', 'CNAE 95 Classe', 'Distritos SP', 'Qtd Vínculos CLT', 'Qtd Vínculos Ativos', 'Qtd Vínculos Estatutários', 'Ind Atividade Ano', 'Ind CEI Vinculado', 'Ind Estab Participa PAT', 'Ind Rais Negativa', 'Ind Simples', 'Município', 'Natureza Jurídica', 'Regiões Adm DF', 'CNAE 2.0 Subclasse', 'Tamanho Estabelecimento', 'Tipo Estab', 'Tipo Estab.1', 'Ano']
Colunas do arquivo ESTB2011.csv:
['Bairros SP', 'Bairros Fortaleza', 'Bairros RJ', 'CNAE 2.0 Classe', 'CNAE 95 Classe', 'Distritos SP', 'Qtd Vínculos CLT', 'Qtd Vínculos Ativos', 'Qtd Vínculos Estatutários', 'Ind Atividade Ano', 'Ind CEI Vinculado', 'Ind Estab Participa PAT', 'Ind Rais Negativa', 'Ind Simples', 'Município', 'Natureza Jurídica', 'Regiões Adm DF', 'CNAE 2.0 Subclasse', 'Tamanho Estabelecimento', 'Tipo Estab', 'Tipo Estab.1', 'Ano']
Colunas do arquivo ESTB2012.csv:
['Bairros SP', 'Bairros Fortaleza', 'Bairros RJ', 'CNAE 2.0 Class

C:\Users\thiago.silva\AppData\Local\Temp\ipykernel_23040\2514611392.py:3: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_arquivo, delimiter=';')


Colunas do arquivo ESTB2019.csv:
['Bairros SP', 'Bairros Fortaleza', 'Bairros RJ', 'CNAE 2.0 Classe', 'CNAE 95 Classe', 'Distritos SP', 'Qtd Vínculos CLT', 'Qtd Vínculos Ativos', 'Qtd Vínculos Estatutários', 'Ind Atividade Ano', 'Ind CEI Vinculado', 'Ind Estab Participa PAT', 'Ind Rais Negativa', 'Ind Simples', 'Município', 'Natureza Jurídica', 'Regiões Adm DF', 'CNAE 2.0 Subclasse', 'Tamanho Estabelecimento', 'Tipo Estab', 'Tipo Estab.1', 'UF', 'IBGE Subsetor', 'CEP Estab', 'Ano']


C:\Users\thiago.silva\AppData\Local\Temp\ipykernel_23040\2514611392.py:3: DtypeWarning: Columns (3,4,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_arquivo, delimiter=';')


Colunas do arquivo ESTB2020.csv:
['Bairros SP', 'Bairros Fortaleza', 'Bairros RJ', 'CNAE 2.0 Classe', 'CNAE 95 Classe', 'Distritos SP', 'Qtd Vínculos CLT', 'Qtd Vínculos Ativos', 'Qtd Vínculos Estatutários', 'Ind Atividade Ano', 'Ind CEI Vinculado', 'Ind Estab Participa PAT', 'Ind Rais Negativa', 'Ind Simples', 'Município', 'Natureza Jurídica', 'Regiões Adm DF', 'CNAE 2.0 Subclasse', 'Tamanho Estabelecimento', 'Tipo Estab', 'Tipo Estab.1', 'UF', 'IBGE Subsetor', 'CEP Estab', 'Ano']


C:\Users\thiago.silva\AppData\Local\Temp\ipykernel_23040\2514611392.py:3: DtypeWarning: Columns (3,4,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_arquivo, delimiter=';')


Colunas do arquivo ESTB2021.csv:
['Bairros SP', 'Bairros Fortaleza', 'Bairros RJ', 'CNAE 2.0 Classe', 'CNAE 95 Classe', 'Distritos SP', 'Qtd Vínculos CLT', 'Qtd Vínculos Ativos', 'Qtd Vínculos Estatutários', 'Ind Atividade Ano', 'Ind CEI Vinculado', 'Ind Estab Participa PAT', 'Ind Rais Negativa', 'Ind Simples', 'Município', 'Natureza Jurídica', 'Regiões Adm DF', 'CNAE 2.0 Subclasse', 'Tamanho Estabelecimento', 'Tipo Estab', 'Tipo Estab.1', 'UF', 'IBGE Subsetor', 'CEP Estab', 'Ano']


In [27]:
colunas_por_ano = {}

for arquivo in os.listdir(path):
    if arquivo.startswith('ESTB') and arquivo.endswith(".csv"):
        ano = arquivo[4:8]
        caminho_arquivo = os.path.join(path, arquivo)
        
        df = pd.read_csv(caminho_arquivo, delimiter=';')
        colunas = df.columns.tolist()
        
        colunas_por_ano[f"colunas_{ano}"] = colunas

C:\Users\thiago.silva\AppData\Local\Temp\ipykernel_21984\367050420.py:8: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_arquivo, delimiter=';')
C:\Users\thiago.silva\AppData\Local\Temp\ipykernel_21984\367050420.py:8: DtypeWarning: Columns (3,4,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_arquivo, delimiter=';')
C:\Users\thiago.silva\AppData\Local\Temp\ipykernel_21984\367050420.py:8: DtypeWarning: Columns (3,4,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_arquivo, delimiter=';')


In [28]:
colunas_2010 = colunas_por_ano["colunas_2010"]
colunas_2011 = colunas_por_ano["colunas_2011"]
colunas_2012 = colunas_por_ano["colunas_2012"]
colunas_2013 = colunas_por_ano["colunas_2013"]
colunas_2014 = colunas_por_ano["colunas_2014"]
colunas_2015 = colunas_por_ano["colunas_2015"]
colunas_2016 = colunas_por_ano["colunas_2016"]
colunas_2017 = colunas_por_ano["colunas_2017"]
colunas_2018 = colunas_por_ano["colunas_2018"]
colunas_2019 = colunas_por_ano["colunas_2019"]
colunas_2020 = colunas_por_ano["colunas_2020"]
colunas_2021 = colunas_por_ano["colunas_2021"]

In [32]:
resultado = list(set(colunas_2012) ^ set(colunas_2013))
resultado

['UF']

In [33]:
resultado = list(set(colunas_2013) ^ set(colunas_2014))
resultado

['CEP Estab', 'IBGE Subsetor']

In [36]:
for ano in range(2010, 2015):
    nome_arquivo = f'ESTB{ano}.csv'
    caminho_arquivo = os.path.join(path, nome_arquivo)
    
    if os.path.exists(caminho_arquivo):
        df = pd.read_csv(caminho_arquivo, delimiter=';', encoding='cp1252')
        df['IBGE Subsetor'] = None
        df['CEP Estab'] = None
        
        if ano < 2014:
            df['UF'] = None
        
        df.to_csv(caminho_arquivo, sep=';', index=False, encoding='cp1252')
        print(f"Colunas faltantes adicionadas ao arquivo {nome_arquivo}.")
    else:
        print(f"Arquivo {nome_arquivo} não encontrado.")

print("Conclusão: Colunas faltantes adicionadas em todos os arquivos CSV.")

Colunas faltantes adicionadas ao arquivo ESTB2010.csv.
Colunas faltantes adicionadas ao arquivo ESTB2011.csv.
Colunas faltantes adicionadas ao arquivo ESTB2012.csv.
Colunas faltantes adicionadas ao arquivo ESTB2013.csv.
Colunas faltantes adicionadas ao arquivo ESTB2014.csv.
Conclusão: Colunas faltantes adicionadas em todos os arquivos CSV.


In [37]:
ordem_colunas = colunas_2021

for ano in range(2010, 2015):
    nome_arquivo = f'ESTB{ano}.csv'
    caminho_arquivo = os.path.join(path, nome_arquivo)

    if os.path.exists(caminho_arquivo):
        df = pd.read_csv(caminho_arquivo, delimiter=';', encoding='cp1252')
        df = df.reindex(columns=ordem_colunas)
        df.to_csv(caminho_arquivo, sep=';', index=False)
        print(f"Colunas ordenadas no arquivo {nome_arquivo}.")
    else:
        print(f"Arquivo {nome_arquivo} não encontrado.")

print("Conclusão: Colunas ordenadas em todos os arquivos CSV.")

Colunas ordenadas no arquivo ESTB2010.csv.
Colunas ordenadas no arquivo ESTB2011.csv.
Colunas ordenadas no arquivo ESTB2012.csv.
Colunas ordenadas no arquivo ESTB2013.csv.
Colunas ordenadas no arquivo ESTB2014.csv.
Conclusão: Colunas ordenadas em todos os arquivos CSV.


In [38]:
resultado = list(set(colunas_2012) ^ set(colunas_2013))
resultado

['UF']

In [39]:
resultado = list(set(colunas_2013) ^ set(colunas_2014))
resultado

['CEP Estab', 'IBGE Subsetor']

In [40]:
resultado = list(set(colunas_2021) ^ set(colunas_2010))
resultado

['CEP Estab', 'UF', 'IBGE Subsetor']

In [41]:
#Informações de conexão com o banco Postgres
host='dataiesb.iesbtech.com.br'
dbname='IESB_Empresas'
user='1922120026_TCC_Thiago'
password='*********'

In [42]:
# Função para criar conexão no banco
def conecta_db():
    con = psycopg2.connect(host=host, 
                         database=dbname,
                         user=user, 
                         password=password)
    return con

def postgres_test():
    try:
        con = conecta_db()
        con.close()
        print("Conexão com o banco de dados realizada com sucesso!")
    except:
        print("A conexão com o banco de dados falhou!")

postgres_test()

Conexão com o banco de dados realizada com sucesso!


In [49]:
# Função para criar tabela no banco
def criar_db(sql):
    con = conecta_db()
    cur = con.cursor()
    cur.execute(sql)
    con.commit()
    con.close()

In [50]:
# Criando a tabela para os dados das RAIS ESTABELECIMENTO
sql = '''CREATE TABLE IF NOT EXISTS tb_rais_estabelecimento (
cnae_95_classe varchar(255),
ind_cei_vinculado varchar(255),
ind_simples varchar(255),
ibge_subsetor varchar(255),
distritos_sp varchar(255),
bairros_rj varchar(255),
regioes_adm_df varchar(255),
qtd_vinculos_estatutarios varchar(255),
ind_estab_participa_pat varchar(255),
cep_estab varchar(255),
bairros_sp varchar(255),
qtd_vinculos_ativos varchar(255),
uf varchar(255),
municipio varchar(255),
bairros_fortaleza varchar(255),
cnae_2_0_classe varchar(255),
natureza_juridica varchar(255),
ind_rais_negativa varchar(255),
tipo_estab varchar(255),
tipo_estab_1 varchar(255),
tamanho_estabelecimento varchar(255),
ind_atividade_ano varchar(255),
cnae_2_0_subclasse varchar(255),
ano varchar(255),
qtd_vinculos_clt varchar(255)
  );'''

criar_db(sql)

In [43]:
def copy_from_stringio(con, year, table):
    """
    Here we are going save the dataframe in memory 
    and use copy_from() to copy it to the table
    """
    # save dataframe to an in memory buffer
    buffer = StringIO()
    nome_arquivo = f'ESTB{year}.csv'
    caminho_arquivo = os.path.join(path, nome_arquivo)
    df = pd.read_csv(caminho_arquivo, delimiter=';')
    df.to_csv(buffer, index=False, header=False)
    buffer.seek(0)
    
    cursor = con.cursor()
    try:
        cursor.copy_from(buffer, table, sep=",")
        con.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        con.rollback()
        cursor.close()
        return 1
    print(f"copy_from_stringio() done for {year}")
    cursor.close()

In [51]:
con = conecta_db()
tm.sleep(1)

start = timer()
copy_from_stringio(con, 2010, 'tb_rais_estabelecimento')
finish = timer()
tempo = finish - start
print(f'Arquivo de 2010 incluído no banco em {tempo} segundos.')
tm.sleep(30)

start = timer()
copy_from_stringio(con, 2011, 'tb_rais_estabelecimento')
finish = timer()
tempo = finish - start
print(f'Arquivo de 2011 incluído no banco em {tempo} segundos.')
tm.sleep(30)

start = timer()
copy_from_stringio(con, 2012, 'tb_rais_estabelecimento')
finish = timer()
tempo = finish - start
print(f'Arquivo de 2012 incluído no banco em {tempo} segundos.')
tm.sleep(30)

start = timer()
copy_from_stringio(con, 2013, 'tb_rais_estabelecimento')
finish = timer()
tempo = finish - start
print(f'Arquivo de 2013 incluído no banco em {tempo} segundos.')
tm.sleep(30)

start = timer()
copy_from_stringio(con, 2014, 'tb_rais_estabelecimento')
finish = timer()
tempo = finish - start
print(f'Arquivo de 2014 incluído no banco em {tempo} segundos.')
tm.sleep(30)

start = timer()
copy_from_stringio(con, 2015, 'tb_rais_estabelecimento')
finish = timer()
tempo = finish - start
print(f'Arquivo de 2015 incluído no banco em {tempo} segundos.')
tm.sleep(30)

start = timer()
copy_from_stringio(con, 2016, 'tb_rais_estabelecimento')
finish = timer()
tempo = finish - start
print(f'Arquivo de 2016 incluído no banco em {tempo} segundos.')
tm.sleep(30)

start = timer()
copy_from_stringio(con, 2017, 'tb_rais_estabelecimento')
finish = timer()
tempo = finish - start
print(f'Arquivo de 2017 incluído no banco em {tempo} segundos.')
tm.sleep(30)

start = timer()
copy_from_stringio(con, 2018, 'tb_rais_estabelecimento')
finish = timer()
tempo = finish - start
print(f'Arquivo de 2018 incluído no banco em {tempo} segundos.')
tm.sleep(30)

start = timer()
copy_from_stringio(con, 2019, 'tb_rais_estabelecimento')
finish = timer()
tempo = finish - start
print(f'Arquivo de 2019 incluído no banco em {tempo} segundos.')
tm.sleep(30)

start = timer()
copy_from_stringio(con, 2020, 'tb_rais_estabelecimento')
finish = timer()
tempo = finish - start
print(f'Arquivo de 2020 incluído no banco em {tempo} segundos.')
tm.sleep(30)

start = timer()
copy_from_stringio(con, 2021, 'tb_rais_estabelecimento')
finish = timer()
tempo = finish - start
print(f'Arquivo de 2021 incluído no banco em {tempo} segundos.')

copy_from_stringio() done for 2010
Arquivo de 2010 incluído no banco em 1052.3579628001899 segundos.
copy_from_stringio() done for 2011
Arquivo de 2011 incluído no banco em 751.9692470999435 segundos.
copy_from_stringio() done for 2012
Arquivo de 2012 incluído no banco em 810.372569900006 segundos.
copy_from_stringio() done for 2013
Arquivo de 2013 incluído no banco em 748.4472542998847 segundos.
copy_from_stringio() done for 2014
Arquivo de 2014 incluído no banco em 529.3301528000738 segundos.
copy_from_stringio() done for 2015
Arquivo de 2015 incluído no banco em 748.6352554999758 segundos.
copy_from_stringio() done for 2016
Arquivo de 2016 incluído no banco em 927.6303840999026 segundos.
copy_from_stringio() done for 2017
Arquivo de 2017 incluído no banco em 885.4010523001198 segundos.
copy_from_stringio() done for 2018
Arquivo de 2018 incluído no banco em 763.6537583000027 segundos.


C:\Users\thiago.silva\AppData\Local\Temp\ipykernel_21984\3537777107.py:10: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_arquivo, delimiter=';')


copy_from_stringio() done for 2019
Arquivo de 2019 incluído no banco em 753.461477200035 segundos.


C:\Users\thiago.silva\AppData\Local\Temp\ipykernel_21984\3537777107.py:10: DtypeWarning: Columns (3,4,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_arquivo, delimiter=';')


copy_from_stringio() done for 2020
Arquivo de 2020 incluído no banco em 711.7831147999968 segundos.


C:\Users\thiago.silva\AppData\Local\Temp\ipykernel_21984\3537777107.py:10: DtypeWarning: Columns (3,4,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_arquivo, delimiter=';')


copy_from_stringio() done for 2021
Arquivo de 2021 incluído no banco em 716.8752200999297 segundos.


Aproximadamente 2h e 37 minutos para escrever todos os registros